# Video 2 Recipe

Programa capaz de extraer recetas de cocina a partir de la transcripción del audio de un video de cocina en Youtube. Útil para aquellas recetas que no cuentan con una transcripción en la descripción.

## Imports

In [1]:
import os
import sys

# pip install SpeechRecognition
import speech_recognition as sr

# pip install --upgrade youtube-dl
import youtube_dl as ydl

# pip install webvtt-py
import webvtt

# Para capturar stdout como un string
from io import StringIO

## Video a Procesar

In [2]:
# URL del video del que se desea extraer una receta
video_url = "https://www.youtube.com/watch?v=Vr-o01qiRYI"

In [68]:
# Opciones de descarga:
# - Descargar todos los archivos como "source"
# - Descargar los subtítulos ya dados por el video
# - Obviar la descarga del video como tal.
# - Se descargan los subtítulos del video (oficiales si existen, automáticos si no los hay)
download_options = {
    'outtmpl': './media/subs',
    'writesubtitles': True,
    'writeautomaticsub': True,
    "skip_download": True
}

# Se realiza la descarga
with ydl.YoutubeDL(download_options) as video:
    video.download([video_url])

# Se listan todos los archivos de la carpeta "media"
files = os.listdir("./media")

# Se revisa si algún archivo contiene "subs" en su nombre
# de ser así, se extrae su path.
for file in files:
    if "subs" in file:
        subtitles_path = file
        break

# Se inicializa la string d
previous_caption = ""
caption_snippets = []

for i, caption in enumerate(webvtt.read("./media/" + subtitles_path)):

    caption = caption.text.replace("&nbsp;", " ")
    caption = caption.replace("\n", "")
    caption = caption.strip()

    # Se agrega siempre la primera caption a los snippets
    if i == 0:
        caption_snippets.append(caption)

    # Si la caption anterior es parte de la nueva caption
    # se agrega la caption anterior a los snippets
    elif previous_caption in caption:
        caption_snippets.append(previous_caption)

    # Se actualiza la caption anterior
    previous_caption = caption

# Se eliminan los strings repetidos 
# Se crea un diccionario que utiliza como llaves los strings del corpus
# dado que un diccionario no puede tener llaves repetidas, elimina las
# repetidas y las retorna ordenadas. Se puede hacer la misma operación 
# usando "sets" pero retorna los elementos del corpus desordenados.
caption_snippets = list(dict.fromkeys(caption_snippets))

# Se unen todas las strings
corpus = " ".join(caption_snippets)

[youtube] Vr-o01qiRYI: Downloading webpage
[info] Writing video subtitles to: media\subs.en.vtt


In [69]:
corpus

"[Music]  America I know what you're thinking do we really need a recipe for baked potatoes well here in the Test Kitchen we baked over 200 pounds of spuds to discover that very answer and today I'm here with the expert Ellie who's gonna show us why we do need a recipe Bridgette some crazy things are happening in the world with baked potatoes and it has to stop Oh No immediately first we're cooking our potatoes in the microwave not good I've done it not gonna I've done it's right and it cooks unevenly it cooks from the inside out we also cook our potatoes in foil I've done that too and it traps in all of the moisture and it doesn't give us a tasty potato and finally when we do get it in the oven to bake it we let it hang out on the counter forever and there's no fluffiness coming out of that potato at all but their greatest doorstops absolutely today we're gonna do my favorite thing one of the things we love to do in the Test Kitchen we're gonna brine potatoes we're gonna brine potatoe

## Chequeo Subtítulos

Se solicitan los subtítulos de un video. Si el video no tiene subtítulos oficiales, la bandera "has_subs" se retorna como False.

In [3]:
# Reemplaza el stdout normal por uno custom llamado "mystdout"
old_stdout = sys.stdout
sys.stdout = temp_stdout = StringIO()

# Opciones de descarga:
# - Listar los subtítulos disponibles
# - No descargar el video
download_options = {
    'listsubtitles': True,
    "skip_download": True
}

# Se realiza el request. El progreso del programa es guardado
# en la variable "mystdout"
with ydl.YoutubeDL(download_options) as video:
    video.download([video_url])

# Se vuelve a poner "stdout" como logger
sys.stdout = old_stdout

# Se guardan los logs generados en una variable
ydl_logs = temp_stdout.getvalue()

# Si los logs contienen el string "has no subtitles" se
# setea una variable como "False" para indicar esto.
if "has no subtitles" in ydl_logs:
    has_subtitles = False
    print("El video no tiene subtítulos.")
else:
    has_subtitles = True
    print("El video tiene subtítulos")

El video no tiene subtítulos.


## Transcripción de Audio

Dependiendo de si la bandera "has_subtitles" es True o False, el programa opta por obtener los subtítulos oficiales del video o la transcripción a través del uso de la API de google.

In [7]:
# ======================
# DESCARGA DE SUBTÍTULOS
# ======================

if has_subtitles:

    # Opciones de descarga:
    # - Descargar todos los archivos como "source"
    # - Descargar los subtítulos ya dados por el video
    # - Obviar la descarga del video como tal.
    download_options = {
        'outtmpl': './media/subs',
        'writesubtitles': True,
        'writeauto'
        "skip_download": True
    }

    # Se realiza la descarga
    with ydl.YoutubeDL(download_options) as video:
        video.download([video_url])

    # Se listan todos los archivos de la carpeta "media"
    files = os.listdir("./media")

    # Se revisa si algún archivo contiene "subs" en su nombre
    # de ser así, se extrae su path.
    for file in files:
        if "subs" in file:
            subtitles_path = file
            break

    # Corpus del video
    video_corpus = ""

    # Se extrae el texto del archivo .vtt  
    for caption in webvtt.read("./media/" + subtitles_path):

        # Se eliminan:
        # - Hard spaces (&nbsp;)
        # - Newlines (\n)
        # - Leading and trailing spaces
        caption = caption.text.replace("&nbsp;", " ")
        caption = caption.replace("\n", "")
        caption = caption.strip()

        # Se agrega el string limpio al corpus
        video_corpus = video_corpus + " " + caption

# ======================
# TRANSCRIPCIÓN CON API
# ======================

else:

    # Opciones de descarga:
    # - Descargar todos los archivos como "audio"
    # - Descargar el mejor audio posible
    # - Convertir el audio a wav y transcribirlo con FFMPEG
    download_options = {
        'outtmpl': './media/audio.%(ext)s',
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }]
    }

    # Se realiza la descarga
    with ydl.YoutubeDL(download_options) as video:
        video.download([video_url])

    # Se inicializa la clase de reconocimiento de voz
    recognizer = sr.Recognizer()

    # Lectura del archivo de audio fuente
    with sr.AudioFile('./media/audio.wav') as source:
        audio_text = recognizer.record(source)
        
    # El método de "recognize_()" retorna un error en la request si la
    # API que utiliza "speech recognizer" no se puede accesar. De aquí el "try".
    try:
        # API de reconocimiento de voz de Google
        # Ver la documentación para más lenguajes:
        # https://cloud.google.com/speech-to-text/docs/languages
        video_corpus = recognizer.recognize_google(audio_text, show_all=True)
    
    except Exception as e:
        video_corpus = None
        print('No se consiguió establecer una conexión con la API de Google. Error: {e}')

    # Se imprime el texto generado en caso exista
    if video_corpus:
        print('Conversión exitosa.')


[youtube] Vr-o01qiRYI: Downloading webpage
[download] Destination: media\audio.m4a
[download] 100% of 7.46MiB in 00:01                  
[ffmpeg] Correcting container in "media\audio.m4a"
[ffmpeg] Destination: media\audio.wav
Deleting original file media\audio.m4a (pass -k to keep)
No se consiguió establecer una conexión con la API de Google. Error: {e}


## Visualizando Corpus

In [8]:
print(video_corpus)

None


## Alternativa: IBM Watson Transcription

In [21]:
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import subprocess

# Credenciales de IBM Cloud
api_key = "sD87-ETvf4Va0lJbI2jKZ5nDY_dHccbhxHEWmSdGpoON"
watson_url = "https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/17c9cba9-292d-457d-ab2f-be4bc368e0c8/"

# Se autentica la  conexión con IBM Cloud
authenticator = IAMAuthenticator(api_key)
speech2text = SpeechToTextV1(authenticator = authenticator)
speech2text.set_service_url(watson_url)

# Se rompe el audio en trozos de 6 minutos (360 s)
command = "ffmpeg -i media/audio.mp3 -f segment -segment_time 360 -c copy media/sliced_audio/%03d.mp3"
subprocess.call(command, shell=True)

# Se obtienen los nombres de todos los audios sliceados
files = ["./media/sliced_audio/" + file for file in os.listdir("./media/sliced_audio") if file.endswith(".mp3")]

# Se ordenan los nombres de forma ascendente
files.sort()

# La lista de resultados inicia vacía
results = []

# Se realiza la transcripción
for filename in files:
    with open(filename, "rb") as f:

        # Opciones:
        # - Se va a procesar el archivo "f"
        # - El archivo es un mp3
        # - Se utilizará el modelo para audio estadounidense (US)
        # - Se alimentarán varios audios de manera continua
        # - El timeout máximo será igual al largo en minutos de cada audio.
        response = speech2text.recognize(audio=f, content_type="audio/mp3", model="en-US_NarrowbandModel", inactivity_timeout=360).get_result()
        
        # Se agregan los resultados a la lista
        results.append(response)

# String inicial vacío para el corpus
corpus = ''

# Se construye el corpus del audio
for file in results:
    for result in file["results"]:
        corpus = corpus + result["alternatives"][0]["transcript"].strip() + ". "